# 03 - Toxicity Analysis

Tag trades with conditions and analyze which conditions lead to poor forward returns.

**Toxicity Conditions:**
- **Adverse Selection**: Opponent's first-level size too large -> we may be picked off
- **Information Leakage**: Opponent's first-level size too small -> informed trader hiding
- **Wide Spread**: Large bid-ask spread indicates uncertainty
- **Touch Imbalance**: Extreme imbalance in top-of-book

**Goal**: Identify conditions where our alpha predictions are unreliable.

**VizFlow v0.5.0 Note:**
Zero prices return `null` instead of `inf` for y_* columns.

In [14]:
import polars as pl
import vizflow as vf
from pathlib import Path

print(f"VizFlow version: {vf.__version__}")

VizFlow version: 0.5.0


In [15]:
# Load config
import sys
sys.path.insert(0, str(Path.cwd().parent / "configs"))
from default import config

vf.set_config(config)

## 1. Load Data with Forward Returns

In [ ]:
DATE = "11110101"  # Use same date for trade and alpha

# Load and prepare trade data
df_trade = vf.scan_trade(DATE)
df_trade = vf.parse_time(df_trade, timestamp_col="alpha_ts")
df_trade = df_trade.with_columns([
    ((pl.col("bid_px0") + pl.col("ask_px0")) / 2).alias("mid"),
    (pl.col("ask_px0") - pl.col("bid_px0")).alias("spread"),
    # Spread in bps
    ((pl.col("ask_px0") - pl.col("bid_px0")) / ((pl.col("bid_px0") + pl.col("ask_px0")) / 2) * 10000).alias("spread_bps"),
])

# Load and prepare alpha data (same date)
df_alpha = vf.scan_alpha(DATE)
df_alpha = vf.parse_time(df_alpha, timestamp_col="ticktime")
df_alpha = df_alpha.with_columns(
    ((pl.col("bid_px0") + pl.col("ask_px0")) / 2).alias("mid")
)

# Calculate forward returns
HORIZONS = [60, 180, 1800]  # 60s, 3m, 30m
df = vf.forward_return(
    df_trade,
    df_alpha,
    horizons=HORIZONS,
    trade_time_col="elapsed_alpha_ts",
    alpha_time_col="elapsed_ticktime",
    price_col="mid",
    symbol_col="ukey",
)

df = df.collect()
print(f"Total trades: {len(df):,}")

## 2. Define Toxicity Conditions

Tag trades based on market conditions at execution time.

In [17]:
# Calculate condition metrics
# Column names from presets: bid_size0, ask_size0 (not bid_qty0/ask_qty0)

df = df.with_columns([
    # Top-of-book imbalance: (bid_size - ask_size) / (bid_size + ask_size)
    # Positive = more on bid side, Negative = more on ask side
    (
        (pl.col("bid_size0") - pl.col("ask_size0")) / 
        (pl.col("bid_size0") + pl.col("ask_size0"))
    ).alias("touch_imbalance"),
    
    # Opponent size (opposite side of our trade)
    pl.when(pl.col("order_side") == "Buy")
        .then(pl.col("ask_size0"))
        .otherwise(pl.col("bid_size0"))
        .alias("opponent_size"),
])

print("Condition metrics added")
print(df.select(["spread_bps", "touch_imbalance", "opponent_size"]).describe())

Condition metrics added
shape: (9, 4)
┌────────────┬────────────┬─────────────────┬───────────────┐
│ statistic  ┆ spread_bps ┆ touch_imbalance ┆ opponent_size │
│ ---        ┆ ---        ┆ ---             ┆ ---           │
│ str        ┆ f64        ┆ f64             ┆ f64           │
╞════════════╪════════════╪═════════════════╪═══════════════╡
│ count      ┆ 318.0      ┆ 318.0           ┆ 318.0         │
│ null_count ┆ 0.0        ┆ 0.0             ┆ 0.0           │
│ mean       ┆ 17.104034  ┆ 0.012557        ┆ 25851.36478   │
│ std        ┆ 10.212137  ┆ 0.425407        ┆ 14165.221155  │
│ min        ┆ 4.414037   ┆ -0.920789       ┆ 1077.0        │
│ 25%        ┆ 8.920607   ┆ -0.272756       ┆ 14565.0       │
│ 50%        ┆ 13.44086   ┆ 0.004429        ┆ 25701.0       │
│ 75%        ┆ 27.434842  ┆ 0.307745        ┆ 38292.0       │
│ max        ┆ 42.342978  ┆ 0.908481        ┆ 49973.0       │
└────────────┴────────────┴─────────────────┴───────────────┘


In [18]:
# Define thresholds for toxicity conditions
# NOTE: These are example thresholds - calibrate based on your data

SPREAD_HIGH_BPS = 5.0       # Wide spread threshold (bps)
OPPONENT_LARGE = 50000      # Large opponent size threshold
OPPONENT_SMALL = 1000       # Small opponent size threshold
IMBALANCE_EXTREME = 0.7     # Extreme imbalance threshold (abs value)

# Tag trades with toxicity conditions
df = df.with_columns([
    # Wide spread - high uncertainty
    (pl.col("spread_bps") > SPREAD_HIGH_BPS).alias("tox_wide_spread"),
    
    # Large opponent - potential adverse selection
    (pl.col("opponent_size") > OPPONENT_LARGE).alias("tox_adverse_selection"),
    
    # Small opponent - potential information leakage (informed trader hiding)
    (pl.col("opponent_size") < OPPONENT_SMALL).alias("tox_info_leakage"),
    
    # Extreme touch imbalance
    (pl.col("touch_imbalance").abs() > IMBALANCE_EXTREME).alias("tox_imbalance"),
])

# Summary of toxicity flags
tox_cols = [c for c in df.columns if c.startswith("tox_")]
print("Toxicity condition rates:")
for col in tox_cols:
    rate = df[col].sum() / len(df) * 100
    print(f"  {col}: {rate:.2f}%")

Toxicity condition rates:
  tox_wide_spread: 93.71%
  tox_adverse_selection: 0.00%
  tox_info_leakage: 0.00%
  tox_imbalance: 11.95%


## 3. Analyze Forward Returns by Toxicity

Compare forward returns for toxic vs non-toxic trades.

In [19]:
def analyze_toxicity(df: pl.DataFrame, tox_col: str, return_col: str = "y_60s"):
    """Analyze forward returns for toxic vs non-toxic trades."""
    result = df.group_by(tox_col).agg([
        pl.len().alias("count"),
        pl.col(return_col).mean().alias("mean_return"),
        pl.col(return_col).std().alias("std_return"),
        pl.col(return_col).median().alias("median_return"),
    ]).sort(tox_col)
    
    return result


# Analyze each toxicity condition
print("=" * 60)
for tox_col in tox_cols:
    print(f"\n{tox_col}:")
    result = analyze_toxicity(df, tox_col, "y_60s")
    print(result)
    print("-" * 40)


tox_wide_spread:
shape: (2, 5)
┌─────────────────┬───────┬─────────────┬────────────┬───────────────┐
│ tox_wide_spread ┆ count ┆ mean_return ┆ std_return ┆ median_return │
│ ---             ┆ ---   ┆ ---         ┆ ---        ┆ ---           │
│ bool            ┆ u32   ┆ f64         ┆ f64        ┆ f64           │
╞═════════════════╪═══════╪═════════════╪════════════╪═══════════════╡
│ false           ┆ 20    ┆ -0.027414   ┆ 0.046791   ┆ -0.022366     │
│ true            ┆ 298   ┆ -0.020648   ┆ 0.025823   ┆ -0.01899      │
└─────────────────┴───────┴─────────────┴────────────┴───────────────┘
----------------------------------------

tox_adverse_selection:
shape: (1, 5)
┌───────────────────────┬───────┬─────────────┬────────────┬───────────────┐
│ tox_adverse_selection ┆ count ┆ mean_return ┆ std_return ┆ median_return │
│ ---                   ┆ ---   ┆ ---         ┆ ---        ┆ ---           │
│ bool                  ┆ u32   ┆ f64         ┆ f64        ┆ f64           │
╞════════════

In [20]:
# Analyze across multiple horizons
print("Forward return comparison (mean) by toxicity condition:")
print("=" * 70)

for tox_col in tox_cols:
    print(f"\n{tox_col}:")
    result = df.group_by(tox_col).agg([
        pl.len().alias("n"),
        (pl.col("y_60s").mean() * 10000).alias("y_60s_bps"),
        (pl.col("y_3m").mean() * 10000).alias("y_3m_bps"),
        (pl.col("y_30m").mean() * 10000).alias("y_30m_bps"),
    ]).sort(tox_col)
    print(result)

Forward return comparison (mean) by toxicity condition:

tox_wide_spread:
shape: (2, 5)
┌─────────────────┬─────┬─────────────┬─────────────┬─────────────┐
│ tox_wide_spread ┆ n   ┆ y_60s_bps   ┆ y_3m_bps    ┆ y_30m_bps   │
│ ---             ┆ --- ┆ ---         ┆ ---         ┆ ---         │
│ bool            ┆ u32 ┆ f64         ┆ f64         ┆ f64         │
╞═════════════════╪═════╪═════════════╪═════════════╪═════════════╡
│ false           ┆ 20  ┆ -274.143543 ┆ -288.526057 ┆ -436.144039 │
│ true            ┆ 298 ┆ -206.482201 ┆ -197.728974 ┆ -184.866335 │
└─────────────────┴─────┴─────────────┴─────────────┴─────────────┘

tox_adverse_selection:
shape: (1, 5)
┌───────────────────────┬─────┬─────────────┬─────────────┬────────────┐
│ tox_adverse_selection ┆ n   ┆ y_60s_bps   ┆ y_3m_bps    ┆ y_30m_bps  │
│ ---                   ┆ --- ┆ ---         ┆ ---         ┆ ---        │
│ bool                  ┆ u32 ┆ f64         ┆ f64         ┆ f64        │
╞═══════════════════════╪═════╪═══════

## 4. Combined Toxicity Score

In [21]:
# Count number of toxicity flags per trade
df = df.with_columns(
    (pl.col("tox_wide_spread").cast(pl.Int32)
     + pl.col("tox_adverse_selection").cast(pl.Int32)
     + pl.col("tox_info_leakage").cast(pl.Int32)
     + pl.col("tox_imbalance").cast(pl.Int32)
    ).alias("tox_score")
)

# Analyze by toxicity score
print("Forward returns by toxicity score (0 = clean, 4 = most toxic):")
result = df.group_by("tox_score").agg([
    pl.len().alias("n"),
    (pl.col("y_60s").mean() * 10000).alias("y_60s_bps"),
    (pl.col("y_3m").mean() * 10000).alias("y_3m_bps"),
    (pl.col("y_30m").mean() * 10000).alias("y_30m_bps"),
]).sort("tox_score")

print(result)

Forward returns by toxicity score (0 = clean, 4 = most toxic):
shape: (3, 5)
┌───────────┬─────┬─────────────┬─────────────┬─────────────┐
│ tox_score ┆ n   ┆ y_60s_bps   ┆ y_3m_bps    ┆ y_30m_bps   │
│ ---       ┆ --- ┆ ---         ┆ ---         ┆ ---         │
│ i32       ┆ u32 ┆ f64         ┆ f64         ┆ f64         │
╞═══════════╪═════╪═════════════╪═════════════╪═════════════╡
│ 0         ┆ 18  ┆ -274.143543 ┆ -288.526057 ┆ -436.144039 │
│ 1         ┆ 264 ┆ -191.310699 ┆ -207.66808  ┆ -182.352114 │
│ 2         ┆ 36  ┆ -297.511212 ┆ 60.687795   ┆ -232.636548 │
└───────────┴─────┴─────────────┴─────────────┴─────────────┘


In [22]:
# Binary: toxic vs clean
df = df.with_columns(
    (pl.col("tox_score") > 0).alias("is_toxic")
)

print("\nToxic vs Clean trades:")
result = df.group_by("is_toxic").agg([
    pl.len().alias("n"),
    (pl.len() / len(df) * 100).alias("pct"),
    (pl.col("y_60s").mean() * 10000).alias("y_60s_bps"),
    (pl.col("y_3m").mean() * 10000).alias("y_3m_bps"),
    (pl.col("y_30m").mean() * 10000).alias("y_30m_bps"),
]).sort("is_toxic")

print(result)


Toxic vs Clean trades:
shape: (2, 6)
┌──────────┬─────┬───────────┬─────────────┬─────────────┬─────────────┐
│ is_toxic ┆ n   ┆ pct       ┆ y_60s_bps   ┆ y_3m_bps    ┆ y_30m_bps   │
│ ---      ┆ --- ┆ ---       ┆ ---         ┆ ---         ┆ ---         │
│ bool     ┆ u32 ┆ f64       ┆ f64         ┆ f64         ┆ f64         │
╞══════════╪═════╪═══════════╪═════════════╪═════════════╪═════════════╡
│ false    ┆ 18  ┆ 5.660377  ┆ -274.143543 ┆ -288.526057 ┆ -436.144039 │
│ true     ┆ 300 ┆ 94.339623 ┆ -206.482201 ┆ -197.728974 ┆ -184.866335 │
└──────────┴─────┴───────────┴─────────────┴─────────────┴─────────────┘


## 5. Toxicity by Order Side

Check if toxicity affects buys and sells differently.

In [23]:
print("Toxicity analysis by order side:")
result = df.group_by(["order_side", "is_toxic"]).agg([
    pl.len().alias("n"),
    (pl.col("y_60s").mean() * 10000).alias("y_60s_bps"),
    (pl.col("y_3m").mean() * 10000).alias("y_3m_bps"),
]).sort(["order_side", "is_toxic"])

print(result)

Toxicity analysis by order side:
shape: (4, 5)
┌────────────┬──────────┬─────┬─────────────┬─────────────┐
│ order_side ┆ is_toxic ┆ n   ┆ y_60s_bps   ┆ y_3m_bps    │
│ ---        ┆ ---      ┆ --- ┆ ---         ┆ ---         │
│ str        ┆ bool     ┆ u32 ┆ f64         ┆ f64         │
╞════════════╪══════════╪═════╪═════════════╪═════════════╡
│ Buy        ┆ false    ┆ 9   ┆ 166.481687  ┆ null        │
│ Buy        ┆ true     ┆ 128 ┆ -167.10705  ┆ -224.086756 │
│ Sell       ┆ false    ┆ 9   ┆ -494.456158 ┆ -288.526057 │
│ Sell       ┆ true     ┆ 172 ┆ -240.607331 ┆ -182.224396 │
└────────────┴──────────┴─────┴─────────────┴─────────────┘


## 6. Key Findings Summary

In [24]:
# Summary statistics
total = len(df)
toxic_count = df["is_toxic"].sum()
clean_count = total - toxic_count

print("=" * 60)
print("TOXICITY ANALYSIS SUMMARY")
print("=" * 60)
print(f"\nTotal trades: {total:,}")
print(f"Clean trades: {clean_count:,} ({clean_count/total*100:.1f}%)")
print(f"Toxic trades: {toxic_count:,} ({toxic_count/total*100:.1f}%)")

print("\nCondition breakdown:")
for col in tox_cols:
    count = df[col].sum()
    print(f"  {col}: {count:,} ({count/total*100:.1f}%)")

print("\n" + "=" * 60)

TOXICITY ANALYSIS SUMMARY

Total trades: 318
Clean trades: 18 (5.7%)
Toxic trades: 300 (94.3%)

Condition breakdown:
  tox_wide_spread: 298 (93.7%)
  tox_adverse_selection: 0 (0.0%)
  tox_info_leakage: 0 (0.0%)
  tox_imbalance: 38 (11.9%)



## Next Steps

Based on findings:

1. **Threshold Calibration**: Adjust thresholds based on your data distribution
2. **Feature Engineering**: Add more toxicity conditions if needed
3. **Model Integration**: Use toxicity flags to filter unreliable predictions
4. **VizFlow Enhancement**: If patterns are useful, add `vf.tag_condition()` to VizFlow

**Record findings in FEEDBACK.md** for VizFlow improvements.